# Renewable Energy Park - BESS Simulation & Analysis

## Introduction

This notebook provides a comprehensive simulation and analysis of a generic **Renewable Energy Park** private wire connection to an industrial park + hyperscale datacenter.

### Project Overview

**We're simulating** a hybrid renewable energy installation consisting of:
- **Wind Farm**: 21 turbines × 8 MW = 168 MW total capacity
- **Solar Farm**: ~190 MW total capacity
- **Private Wire Transmission**: High-voltage transmission line connecting the renewable park to the industrial datacenter
- **Battery Energy Storage System (BESS)**: Optional energy storage to optimize power delivery and minimize curtailment

### Private Wire & PPA Structure

The renewable energy park is connected to the industrial datacenter via a **private wire** (dedicated transmission line). Key assumptions:

- **PPA Price**: €50/MWh for all renewable energy delivered via the private wire
- **Net Metering**: All renewable energy delivered to the industrial park is purchased at a fixed PPA price
- **Curtailment**: Energy that cannot be transmitted due to wire capacity constraints is curtailed (wasted)
- **Grid Import**: When renewable generation is insufficient, the datacenter imports from the grid
- **Aux Power**: The park is allowed to import the energy from the Datacenter when the renewables are not available and need auxilliary power to operate

### Simulation Objectives

This notebook simulates two scenarios:

1. **No BESS (Baseline)**: Direct connection with no battery storage
   - Shows curtailment and grid import without storage
   - Establishes baseline economics

2. **With BESS**: Optimized battery dispatch to minimize curtailment
   - Battery charges when generation exceeds wire capacity + load
   - Battery discharges to reduce grid import when generation is insufficient
   - Maximizes renewable energy delivery and PPA revenue

### Key Performance Metrics

- **Curtailment Rate**: Percentage of generated energy that cannot be delivered
- **Grid Import Rate**: Percentage of load met by grid (vs. renewable)
- **Renewable Fraction**: Percentage of load served by renewable energy
- **Wire Utilization**: Average capacity utilization of the private wire
- **BESS Cycles**: Total equivalent full cycles over the simulation period
- **Economic Metrics**: CAPEX, revenue, payback period

## Setup & Installation

This section sets up the Colab environment by:
1. Cloning the simulation code from GitHub
2. Installing required dependencies
3. Mounting Google Drive for data access

In [ ]:
# Clone the GitHub repository
!git clone https://github.com/bessvaluestack/wind-solar-bess-private-wire-sizing.git
%cd wind-solar-bess-private-wire-sizing

In [ ]:
# Install dependencies
!pip install -q -r requirements.txt

In [ ]:
# Mount Google Drive to access data files
from google.colab import drive
drive.mount('/content/drive')

# Set the path to your Google Drive folder containing the data
# IMPORTANT: Update this path to match your Google Drive structure
GDRIVE_DATA_PATH = '/content/drive/MyDrive/Park/data'

print(f"Google Drive mounted. Data will be loaded from: {GDRIVE_DATA_PATH}")

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yaml
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import simulation modules
from src.simulator import SystemSimulator
from src.load_generator import DatacenterLoadGenerator

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("All imports successful!")

## 1. Data Connectivity & Loading

### Expected Data Format

The simulator expects three types of CSV files in your Google Drive folder:

#### 1.1 Wind Generation Data
- **Filename**: `wind_generation_11_years.csv`
- **Columns**: 
  - `Interval_Date_Time` or `timestamp`: Timezone-aware datetime
  - Generation column: Wind generation in kWh (for 15-minute intervals)
- **Resolution**: 15-minute intervals
- **Duration**: 11 years of historical or simulated data

#### 1.2 Solar Generation Data
- **Filename**: `solar_generation_11_years.csv`
- **Columns**: 
  - `Interval_Date_Time` or `timestamp`: Timezone-aware datetime
  - Generation column: Solar generation in kWh (for 15-minute intervals)
- **Resolution**: 15-minute intervals
- **Duration**: Must match wind data duration

#### 1.3 Auxiliary Load Data (Optional)
- **Filename**: User-configurable (or auto-generated)
- **Columns**: 
  - `timestamp`: Timezone-aware datetime
  - `total_aux_mw` or `total_aux_kw`: Auxiliary load consumption
- **Purpose**: Models parasitic loads (inverters, turbine control systems, etc.)

### Load Profile Options

The simulator supports two modes for datacenter load:

1. **Variable Load Profile**: 
   - Use a specific CSV file with actual datacenter load data
   - Allows realistic modeling of datacenter consumption patterns
   - Format: `timestamp`, `load_mw` columns

2. **Assume All Sellable**:
   - Creates a flat load profile at the wire capacity (e.g., 80 MW constant)
   - Represents a "merchant" scenario where all delivered energy can be sold
   - Useful for understanding maximum revenue potential

In [ ]:
# Load the configuration file
with open('config.yaml', 'r') as f:
    config = yaml.safe_load(f)

print("Configuration loaded successfully!")
print("\nKey Configuration Parameters:")
print(f"  Wind Generation File: {config['generation_data']['wind_file']}")
print(f"  Solar Generation File: {config['generation_data']['solar_file']}")
print(f"  Datacenter Peak Load: {config['datacenter']['annual_peak_mw']} MW")
print(f"  Private Wire Capacity: {config['private_wire']['capacity_mw']} MW")
print(f"  BESS Energy Capacity: {config['bess']['energy_capacity_mwh']} MWh")
print(f"  BESS Power Capacity: {config['bess']['power_capacity_mw']} MW")
print(f"  PPA Price: €{config['economics']['ppa_price_eur_per_mwh']}/MWh")

In [ ]:
# Update config to point to Google Drive data
# You can override these paths if your data is stored differently
config['generation_data']['wind_file'] = f"{GDRIVE_DATA_PATH}/wind_generation_11_years.csv"
config['generation_data']['solar_file'] = f"{GDRIVE_DATA_PATH}/solar_generation_11_years.csv"

# Optional: Update load file path if you have datacenter load data
# config['datacenter']['load_file'] = f"{GDRIVE_DATA_PATH}/63_MW_datacenter_load_11_years.csv"
# If not specified, the simulator will generate a synthetic load profile

# Save updated config to a temporary file
with open('config_colab.yaml', 'w') as f:
    yaml.dump(config, f)

print("Configuration updated for Google Drive paths")

In [ ]:
# Initialize simulator and load generation data
simulator = SystemSimulator('config_colab.yaml')

print("Loading generation data from Google Drive...")
generation_df = simulator.load_generation_data()

print(f"\nGeneration data loaded successfully!")
print(f"  Total timesteps: {len(generation_df):,}")
print(f"  Date range: {generation_df.index[0]} to {generation_df.index[-1]}")
print(f"  Duration: {len(generation_df) / (4 * 24 * 365):.1f} years")
print(f"\nFirst few rows:")
print(generation_df.head())

In [ ]:
# Calculate and display generation statistics
timestep_hours = config['simulation']['timestep_minutes'] / 60.0

wind_stats = {
    'Mean (MW)': generation_df['wind_mw'].mean(),
    'Max (MW)': generation_df['wind_mw'].max(),
    'Min (MW)': generation_df['wind_mw'].min(),
    'Std Dev (MW)': generation_df['wind_mw'].std(),
    'Total Energy (GWh)': (generation_df['wind_mw'].sum() * timestep_hours) / 1000,
    'Capacity Factor': generation_df['wind_mw'].mean() / 168  # Assuming 168 MW installed
}

solar_stats = {
    'Mean (MW)': generation_df['solar_mw'].mean(),
    'Max (MW)': generation_df['solar_mw'].max(),
    'Min (MW)': generation_df['solar_mw'].min(),
    'Std Dev (MW)': generation_df['solar_mw'].std(),
    'Total Energy (GWh)': (generation_df['solar_mw'].sum() * timestep_hours) / 1000,
    'Capacity Factor': generation_df['solar_mw'].mean() / generation_df['solar_mw'].max()
}

total_stats = {
    'Mean (MW)': generation_df['total_generation_mw'].mean(),
    'Max (MW)': generation_df['total_generation_mw'].max(),
    'Min (MW)': generation_df['total_generation_mw'].min(),
    'Total Energy (GWh)': (generation_df['total_generation_mw'].sum() * timestep_hours) / 1000
}

print("\n" + "="*60)
print("GENERATION DATA STATISTICS")
print("="*60)

print("\nWind Generation:")
for key, value in wind_stats.items():
    if 'Capacity Factor' in key:
        print(f"  {key:20s}: {value:>10.1%}")
    else:
        print(f"  {key:20s}: {value:>10.2f}")

print("\nSolar Generation:")
for key, value in solar_stats.items():
    if 'Capacity Factor' in key:
        print(f"  {key:20s}: {value:>10.1%}")
    else:
        print(f"  {key:20s}: {value:>10.2f}")

print("\nTotal Generation:")
for key, value in total_stats.items():
    print(f"  {key:20s}: {value:>10.2f}")

print("\n" + "="*60)

### Visualization 1.1: Generation Time Series (Sample Week)

In [ ]:
# Plot one week of generation data
week_data = generation_df.iloc[:4*24*7]  # First week (4 timesteps/hour * 24 hours * 7 days)

fig, ax = plt.subplots(figsize=(14, 6))
ax.plot(week_data.index, week_data['wind_mw'], label='Wind', alpha=0.8, linewidth=1.5)
ax.plot(week_data.index, week_data['solar_mw'], label='Solar', alpha=0.8, linewidth=1.5)
ax.plot(week_data.index, week_data['total_generation_mw'], label='Total', alpha=0.9, linewidth=2, color='black')
ax.axhline(y=config['private_wire']['capacity_mw'], color='red', linestyle='--', label='Wire Capacity', linewidth=2)

ax.set_xlabel('Date/Time', fontsize=12)
ax.set_ylabel('Power (MW)', fontsize=12)
ax.set_title('Renewable Generation - Sample Week', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', fontsize=10)
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Visualization 1.2: Generation Distribution & Correlation

In [ ]:
# Distribution histograms and correlation
fig, axes = plt.subplots(1, 3, figsize=(16, 4))

# Wind distribution
axes[0].hist(generation_df['wind_mw'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0].axvline(x=generation_df['wind_mw'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0].set_xlabel('Wind Power (MW)', fontsize=11)
axes[0].set_ylabel('Frequency', fontsize=11)
axes[0].set_title('Wind Generation Distribution', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Solar distribution
axes[1].hist(generation_df['solar_mw'], bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[1].axvline(x=generation_df['solar_mw'].mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[1].set_xlabel('Solar Power (MW)', fontsize=11)
axes[1].set_ylabel('Frequency', fontsize=11)
axes[1].set_title('Solar Generation Distribution', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

# Wind vs Solar correlation
axes[2].scatter(generation_df['wind_mw'], generation_df['solar_mw'], alpha=0.05, s=1)
axes[2].set_xlabel('Wind Power (MW)', fontsize=11)
axes[2].set_ylabel('Solar Power (MW)', fontsize=11)
axes[2].set_title('Wind vs Solar Correlation', fontsize=12, fontweight='bold')
axes[2].grid(True, alpha=0.3)

# Calculate correlation
correlation = generation_df['wind_mw'].corr(generation_df['solar_mw'])
axes[2].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
             transform=axes[2].transAxes, fontsize=10, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.tight_layout()
plt.show()

### Visualization 1.3: Monthly Generation Patterns

In [ ]:
# Calculate monthly averages
monthly_data = generation_df.copy()
monthly_data['month'] = monthly_data.index.month
monthly_avg = monthly_data.groupby('month')[['wind_mw', 'solar_mw', 'total_generation_mw']].mean()

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(month_names))
width = 0.25

ax.bar(x - width, monthly_avg['wind_mw'], width, label='Wind', alpha=0.8, color='steelblue')
ax.bar(x, monthly_avg['solar_mw'], width, label='Solar', alpha=0.8, color='orange')
ax.bar(x + width, monthly_avg['total_generation_mw'], width, label='Total', alpha=0.8, color='green')

ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Average Power (MW)', fontsize=12)
ax.set_title('Average Monthly Generation', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(month_names)
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## 2. CAPEX Parameters & Economic Configuration

This section allows you to override default CAPEX (capital expenditure) values and economic parameters.

### Default Values from Config:
- **Wind CAPEX**: €200,000,000 (total for 168 MW)
- **Solar CAPEX**: €180,000,000 (total for ~100 MW)
- **BESS CAPEX**: €190,000/MWh
- **Wire CAPEX**: €110,000/MW
- **PPA Price**: €50/MWh
- **Project Lifetime**: 10 years
- **Discount Rate**: 6%

In [ ]:
# CAPEX Override Variables
# Set to None to use config defaults, or specify custom values

# Generation CAPEX (total project costs)
WIND_CAPEX_EUR = None  # e.g., 200_000_000
SOLAR_CAPEX_EUR = None  # e.g., 180_000_000

# BESS CAPEX (per MWh of energy capacity)
BESS_CAPEX_EUR_PER_MWH = None  # e.g., 190_000

# Private Wire CAPEX (per MW of transmission capacity)
WIRE_CAPEX_EUR_PER_MW = None  # e.g., 110_000

# PPA and Financial Parameters
PPA_PRICE_EUR_PER_MWH = None  # e.g., 50
PROJECT_LIFETIME_YEARS = None  # e.g., 10
DISCOUNT_RATE = None  # e.g., 0.06 (6%)

# Apply overrides to config
if WIND_CAPEX_EUR is not None:
    config['economics']['wind_capex_eur'] = WIND_CAPEX_EUR
if SOLAR_CAPEX_EUR is not None:
    config['economics']['solar_capex_eur'] = SOLAR_CAPEX_EUR
if BESS_CAPEX_EUR_PER_MWH is not None:
    config['economics']['bess_capex_eur_per_mwh'] = BESS_CAPEX_EUR_PER_MWH
if WIRE_CAPEX_EUR_PER_MW is not None:
    config['economics']['wire_capex_eur_per_mw'] = WIRE_CAPEX_EUR_PER_MW
if PPA_PRICE_EUR_PER_MWH is not None:
    config['economics']['ppa_price_eur_per_mwh'] = PPA_PRICE_EUR_PER_MWH
if PROJECT_LIFETIME_YEARS is not None:
    config['economics']['project_lifetime_years'] = PROJECT_LIFETIME_YEARS
if DISCOUNT_RATE is not None:
    config['economics']['discount_rate'] = DISCOUNT_RATE

# Save updated config
with open('config_colab.yaml', 'w') as f:
    yaml.dump(config, f)

# Display current economic parameters
print("\n" + "="*60)
print("ECONOMIC PARAMETERS")
print("="*60)
print(f"\nCAPEX (Capital Expenditure):")
print(f"  Wind Farm:           €{config['economics']['wind_capex_eur']:>15,.0f}")
print(f"  Solar Farm:          €{config['economics']['solar_capex_eur']:>15,.0f}")
print(f"  BESS:                €{config['economics']['bess_capex_eur_per_mwh']:>15,.0f} /MWh")
print(f"  Private Wire:        €{config['economics']['wire_capex_eur_per_mw']:>15,.0f} /MW")
print(f"\nRevenue Parameters:")
print(f"  PPA Price:           €{config['economics']['ppa_price_eur_per_mwh']:>15,.2f} /MWh")
print(f"\nFinancial Parameters:")
print(f"  Project Lifetime:    {config['economics']['project_lifetime_years']:>16} years")
print(f"  Discount Rate:       {config['economics']['discount_rate']:>16.1%}")
print("\n" + "="*60)

## 3. No BESS Simulation (Baseline Scenario)

This section simulates the system **without battery storage** to establish a baseline.

### Power Flow Logic (No BESS):
1. **Renewable Generation** → **Auxiliary Loads** (inverters, turbine controls)
2. **Net Available Power** = Generation - Aux Loads
3. **Private Wire Flow** = min(Net Available Power, Wire Capacity)
4. **Curtailment** = max(0, Net Available Power - Wire Capacity)
5. **Grid Import** = max(0, Datacenter Load - Wire Flow)

### Simulation Options:
- **First Year Only**: Fast simulation for testing (set `FIRST_YEAR_ONLY = True`)
- **Multi-Year**: Full simulation period (set `FIRST_YEAR_ONLY = False`)
- **Load Profile**: 
  - Set `LOAD_CSV_PATH` to use a specific datacenter load profile
  - Set `ASSUME_ALL_SELLABLE = True` to maximize revenue (flat load at wire capacity)
  - Leave both as default to generate synthetic datacenter load

In [ ]:
# No BESS Simulation Configuration
FIRST_YEAR_ONLY = False  # Set to True for faster simulation (testing)
ASSUME_ALL_SELLABLE = False  # Set to True for maximum revenue scenario
LOAD_CSV_PATH = None  # Set to path of datacenter load CSV if available
# Example: LOAD_CSV_PATH = f"{GDRIVE_DATA_PATH}/63_MW_datacenter_load_11_years.csv"

print("No BESS Simulation Configuration:")
print(f"  First Year Only: {FIRST_YEAR_ONLY}")
print(f"  Assume All Sellable: {ASSUME_ALL_SELLABLE}")
print(f"  Load CSV Path: {LOAD_CSV_PATH or 'Auto-generate'}")

In [ ]:
# Run No BESS simulation
print("\n" + "="*70)
print("RUNNING NO BESS SIMULATION (BASELINE)")
print("="*70 + "\n")

simulator_no_bess = SystemSimulator('config_colab.yaml')

results_no_bess = simulator_no_bess.run_simulation(
    skip_bess=True,
    first_year_only=FIRST_YEAR_ONLY,
    assume_all_sellable=ASSUME_ALL_SELLABLE,
    load_csv_path=LOAD_CSV_PATH,
    verbose=True
)

# Extract results for easy access
ts_no_bess = results_no_bess['timeseries']
metrics_no_bess = results_no_bess['metrics']

print("\nNo BESS simulation completed!")

### Visualization 3.1: Power Flows (No BESS) - Sample Week

In [ ]:
# Plot power flows for one week
week_data_no_bess = ts_no_bess.iloc[:4*24*7]  # First week

fig, axes = plt.subplots(2, 1, figsize=(14, 10), sharex=True)

# Top panel: Generation and curtailment
axes[0].fill_between(week_data_no_bess['timestamp'], 0, week_data_no_bess['total_generation_mw'], 
                       alpha=0.3, color='green', label='Total Generation')
axes[0].fill_between(week_data_no_bess['timestamp'], 0, week_data_no_bess['curtailment_mw'], 
                       alpha=0.5, color='red', label='Curtailment')
axes[0].plot(week_data_no_bess['timestamp'], week_data_no_bess['wire_flow_mw'], 
             color='blue', linewidth=2, label='Wire Flow')
axes[0].axhline(y=config['private_wire']['capacity_mw'], color='black', 
                linestyle='--', linewidth=2, label='Wire Capacity')
axes[0].set_ylabel('Power (MW)', fontsize=12)
axes[0].set_title('Generation & Wire Flow (No BESS)', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=10)
axes[0].grid(True, alpha=0.3)

# Bottom panel: Load and grid import
axes[1].fill_between(week_data_no_bess['timestamp'], 0, week_data_no_bess['load_mw'], 
                       alpha=0.3, color='purple', label='Datacenter Load')
axes[1].fill_between(week_data_no_bess['timestamp'], 0, week_data_no_bess['grid_import_mw'], 
                       alpha=0.5, color='orange', label='Grid Import')
axes[1].plot(week_data_no_bess['timestamp'], week_data_no_bess['wire_flow_mw'], 
             color='green', linewidth=2, label='Renewable Delivery')
axes[1].set_xlabel('Date/Time', fontsize=12)
axes[1].set_ylabel('Power (MW)', fontsize=12)
axes[1].set_title('Load Coverage (No BESS)', fontsize=14, fontweight='bold')
axes[1].legend(loc='upper right', fontsize=10)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Visualization 3.2: Energy Balance (No BESS)

In [ ]:
# Energy balance Sankey-style visualization
energy_metrics_no_bess = metrics_no_bess['energy']

fig, ax = plt.subplots(figsize=(10, 8))

# Data for plotting
categories = ['Wind', 'Solar', 'Wire\nDelivery', 'Curtailment', 'Load', 'Grid\nImport']
values = [
    energy_metrics_no_bess['total_wind_mwh'] / 1000,  # Convert to GWh
    energy_metrics_no_bess['total_solar_mwh'] / 1000,
    energy_metrics_no_bess['total_wire_delivery_mwh'] / 1000,
    energy_metrics_no_bess['total_curtailment_mwh'] / 1000,
    energy_metrics_no_bess['total_load_mwh'] / 1000,
    energy_metrics_no_bess['total_grid_import_mwh'] / 1000
]
colors = ['steelblue', 'orange', 'green', 'red', 'purple', 'gray']

bars = ax.barh(categories, values, color=colors, alpha=0.7, edgecolor='black')

# Add value labels
for i, (bar, val) in enumerate(zip(bars, values)):
    ax.text(val + max(values)*0.01, i, f'{val:,.0f} GWh', 
            va='center', fontsize=11, fontweight='bold')

ax.set_xlabel('Energy (GWh)', fontsize=12)
ax.set_title('Energy Balance - No BESS Scenario', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

# Print summary
print("\nEnergy Balance Summary (No BESS):")
print(f"  Total Generation: {values[0] + values[1]:,.0f} GWh")
print(f"    → Wire Delivery: {values[2]:,.0f} GWh ({values[2]/(values[0]+values[1])*100:.1f}%)")
print(f"    → Curtailment: {values[3]:,.0f} GWh ({values[3]/(values[0]+values[1])*100:.1f}%)")
print(f"  Total Load: {values[4]:,.0f} GWh")
print(f"    → Renewable: {values[2]:,.0f} GWh ({values[2]/values[4]*100:.1f}%)")
print(f"    → Grid Import: {values[5]:,.0f} GWh ({values[5]/values[4]*100:.1f}%)")

### Visualization 3.3: Duration Curves (No BESS)

In [ ]:
# Duration curves showing sorted power values
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Generation duration curve
gen_sorted = np.sort(ts_no_bess['total_generation_mw'].values)[::-1]
curtail_sorted = np.sort(ts_no_bess['curtailment_mw'].values)[::-1]
wire_sorted = np.sort(ts_no_bess['wire_flow_mw'].values)[::-1]
hours = np.arange(len(gen_sorted)) * (config['simulation']['timestep_minutes'] / 60.0)

axes[0].fill_between(hours, 0, gen_sorted, alpha=0.3, color='green', label='Generation')
axes[0].fill_between(hours, 0, curtail_sorted, alpha=0.5, color='red', label='Curtailment')
axes[0].plot(hours, wire_sorted, color='blue', linewidth=2, label='Wire Flow')
axes[0].axhline(y=config['private_wire']['capacity_mw'], color='black', 
                linestyle='--', linewidth=2, label='Wire Capacity')
axes[0].set_xlabel('Hours', fontsize=12)
axes[0].set_ylabel('Power (MW)', fontsize=12)
axes[0].set_title('Generation Duration Curve', fontsize=13, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Load duration curve
load_sorted = np.sort(ts_no_bess['load_mw'].values)[::-1]
grid_sorted = np.sort(ts_no_bess['grid_import_mw'].values)[::-1]

axes[1].fill_between(hours, 0, load_sorted, alpha=0.3, color='purple', label='Load')
axes[1].fill_between(hours, 0, grid_sorted, alpha=0.5, color='orange', label='Grid Import')
axes[1].plot(hours, wire_sorted, color='green', linewidth=2, label='Renewable Delivery')
axes[1].set_xlabel('Hours', fontsize=12)
axes[1].set_ylabel('Power (MW)', fontsize=12)
axes[1].set_title('Load Duration Curve', fontsize=13, fontweight='bold')
axes[1].legend(loc='upper right', fontsize=9)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 4. With BESS Simulation (Optimized Storage)

This section simulates the system **with battery storage** to minimize curtailment and maximize renewable delivery.

### BESS Dispatch Strategy:
The battery dispatch is optimized using convex optimization (CVXPY) to:
1. **Minimize Curtailment**: Charge battery when generation exceeds wire capacity + load
2. **Minimize Grid Import**: Discharge battery to meet load when generation is insufficient
3. **Respect Constraints**:
   - State of Charge (SoC) limits: 5% - 95%
   - Power limits: Charge/discharge ≤ rated power capacity
   - Daily cycling limits: Prevent excessive battery degradation
   - Wire capacity: Total flow cannot exceed transmission limit

### Power Flow Logic (With BESS):
1. **Renewable Generation** → **Auxiliary Loads**
2. **Net Available Power** = Generation - Aux Loads
3. **BESS Optimization**:
   - If Net Available > Wire Capacity + Load → Charge BESS
   - If Net Available < Load → Discharge BESS (up to wire capacity)
4. **Private Wire Flow** = Generation - Curtailment - BESS Charge + BESS Discharge
5. **Curtailment** = Remaining excess generation (after charging BESS)
6. **Grid Import** = Remaining load deficit (after discharging BESS)

### Configurable Parameters:

In [ ]:
# BESS Simulation Configuration
# Override BESS sizing (set to None to use config defaults)
BESS_ENERGY_MWH = None  # e.g., 200  (battery energy capacity)
BESS_POWER_MW = None    # e.g., 50   (battery power capacity)
WIRE_CAPACITY_MW = None # e.g., 80   (private wire capacity)

# Simulation options
FIRST_YEAR_ONLY_BESS = False  # Set to True for faster simulation
ASSUME_ALL_SELLABLE_BESS = False  # Set to True for maximum revenue scenario
LOAD_CSV_PATH_BESS = None  # Set to datacenter load CSV path if available

print("BESS Simulation Configuration:")
print(f"  BESS Energy: {BESS_ENERGY_MWH or config['bess']['energy_capacity_mwh']} MWh")
print(f"  BESS Power: {BESS_POWER_MW or config['bess']['power_capacity_mw']} MW")
print(f"  Wire Capacity: {WIRE_CAPACITY_MW or config['private_wire']['capacity_mw']} MW")
print(f"  First Year Only: {FIRST_YEAR_ONLY_BESS}")
print(f"  Assume All Sellable: {ASSUME_ALL_SELLABLE_BESS}")
print(f"  Load CSV Path: {LOAD_CSV_PATH_BESS or 'Auto-generate'}")

In [ ]:
# Run BESS simulation
print("\n" + "="*70)
print("RUNNING BESS SIMULATION (OPTIMIZED STORAGE)")
print("="*70 + "\n")

simulator_bess = SystemSimulator('config_colab.yaml')

results_bess = simulator_bess.run_simulation(
    bess_energy_mwh=BESS_ENERGY_MWH,
    bess_power_mw=BESS_POWER_MW,
    wire_capacity_mw=WIRE_CAPACITY_MW,
    skip_bess=False,
    first_year_only=FIRST_YEAR_ONLY_BESS,
    assume_all_sellable=ASSUME_ALL_SELLABLE_BESS,
    load_csv_path=LOAD_CSV_PATH_BESS,
    verbose=True
)

# Extract results
ts_bess = results_bess['timeseries']
metrics_bess = results_bess['metrics']

print("\nBESS simulation completed!")

In [ ]:
# Compare No BESS vs. BESS scenarios
print("\n" + "="*70)
print("SCENARIO COMPARISON: No BESS vs. With BESS")
print("="*70)

comparison = {
    'Curtailment Rate': [
        metrics_no_bess['utilization']['curtailment_rate'],
        metrics_bess['utilization']['curtailment_rate']
    ],
    'Grid Import Rate': [
        metrics_no_bess['utilization']['grid_import_rate'],
        metrics_bess['utilization']['grid_import_rate']
    ],
    'Renewable Fraction': [
        metrics_no_bess['utilization']['renewable_fraction'],
        metrics_bess['utilization']['renewable_fraction']
    ],
    'Wire Utilization': [
        metrics_no_bess['utilization']['wire_utilization'],
        metrics_bess['utilization']['wire_utilization']
    ],
    'Curtailment (GWh)': [
        metrics_no_bess['energy']['total_curtailment_mwh'] / 1000,
        metrics_bess['energy']['total_curtailment_mwh'] / 1000
    ],
    'Grid Import (GWh)': [
        metrics_no_bess['energy']['total_grid_import_mwh'] / 1000,
        metrics_bess['energy']['total_grid_import_mwh'] / 1000
    ],
    'Revenue (M€)': [
        metrics_no_bess['economics']['renewable_revenue_eur'] / 1_000_000,
        metrics_bess['economics']['renewable_revenue_eur'] / 1_000_000
    ]
}

comp_df = pd.DataFrame(comparison, index=['No BESS', 'With BESS']).T
comp_df['Improvement'] = comp_df['With BESS'] - comp_df['No BESS']
comp_df['Improvement %'] = (comp_df['Improvement'] / comp_df['No BESS'].abs()) * 100

print("\n", comp_df.to_string())

print("\n" + "="*70)
print("KEY IMPROVEMENTS:")
curtail_reduction = (metrics_no_bess['energy']['total_curtailment_mwh'] - 
                     metrics_bess['energy']['total_curtailment_mwh']) / 1000
grid_reduction = (metrics_no_bess['energy']['total_grid_import_mwh'] - 
                  metrics_bess['energy']['total_grid_import_mwh']) / 1000
revenue_increase = (metrics_bess['economics']['renewable_revenue_eur'] - 
                    metrics_no_bess['economics']['renewable_revenue_eur']) / 1_000_000

print(f"  Curtailment Reduction: {curtail_reduction:,.0f} GWh ({curtail_reduction/comp_df.loc['Curtailment (GWh)', 'No BESS']*100:.1f}%)")
print(f"  Grid Import Reduction: {grid_reduction:,.0f} GWh ({grid_reduction/comp_df.loc['Grid Import (GWh)', 'No BESS']*100:.1f}%)")
print(f"  Revenue Increase: €{revenue_increase:,.2f}M ({revenue_increase/comp_df.loc['Revenue (M€)', 'No BESS']*100:.1f}%)")
print(f"  BESS Total Cycles: {metrics_bess['bess']['total_cycles']:,.1f}")
print(f"  BESS Degradation: {metrics_bess['bess']['degradation']:.2%}")
print("="*70)

### Visualization 4.1: Power Flows (With BESS) - Sample Week

In [ ]:
# Plot power flows with BESS for one week
week_data_bess = ts_bess.iloc[:4*24*7]  # First week

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)

# Top panel: Generation, curtailment, and BESS
axes[0].fill_between(week_data_bess['timestamp'], 0, week_data_bess['total_generation_mw'], 
                       alpha=0.3, color='green', label='Total Generation')
axes[0].fill_between(week_data_bess['timestamp'], 0, week_data_bess['curtailment_mw'], 
                       alpha=0.5, color='red', label='Curtailment')
axes[0].plot(week_data_bess['timestamp'], week_data_bess['wire_flow_mw'], 
             color='blue', linewidth=2, label='Wire Flow')
axes[0].plot(week_data_bess['timestamp'], week_data_bess['bess_charge_mw'], 
             color='orange', linewidth=1.5, linestyle='--', label='BESS Charge', alpha=0.8)
axes[0].axhline(y=config['private_wire']['capacity_mw'], color='black', 
                linestyle='--', linewidth=2, label='Wire Capacity')
axes[0].set_ylabel('Power (MW)', fontsize=12)
axes[0].set_title('Generation & Wire Flow (With BESS)', fontsize=14, fontweight='bold')
axes[0].legend(loc='upper right', fontsize=9)
axes[0].grid(True, alpha=0.3)

# Middle panel: BESS operation
ax_soc = axes[1].twinx()
axes[1].fill_between(week_data_bess['timestamp'], 0, week_data_bess['bess_charge_mw'], 
                       alpha=0.5, color='orange', label='Charge')
axes[1].fill_between(week_data_bess['timestamp'], 0, -week_data_bess['bess_discharge_mw'], 
                       alpha=0.5, color='purple', label='Discharge')
ax_soc.plot(week_data_bess['timestamp'], week_data_bess['bess_soc'] * 100, 
            color='red', linewidth=2, label='SoC')
axes[1].axhline(y=0, color='black', linewidth=0.5)
axes[1].set_ylabel('BESS Power (MW)', fontsize=12)
ax_soc.set_ylabel('State of Charge (%)', fontsize=12, color='red')
ax_soc.tick_params(axis='y', labelcolor='red')
axes[1].set_title('BESS Operation', fontsize=14, fontweight='bold')
axes[1].legend(loc='upper left', fontsize=9)
ax_soc.legend(loc='upper right', fontsize=9)
axes[1].grid(True, alpha=0.3)

# Bottom panel: Load coverage
axes[2].fill_between(week_data_bess['timestamp'], 0, week_data_bess['load_mw'], 
                       alpha=0.3, color='purple', label='Datacenter Load')
axes[2].fill_between(week_data_bess['timestamp'], 0, week_data_bess['grid_import_mw'], 
                       alpha=0.5, color='orange', label='Grid Import')
axes[2].plot(week_data_bess['timestamp'], week_data_bess['wire_flow_mw'], 
             color='green', linewidth=2, label='Renewable Delivery')
axes[2].set_xlabel('Date/Time', fontsize=12)
axes[2].set_ylabel('Power (MW)', fontsize=12)
axes[2].set_title('Load Coverage (With BESS)', fontsize=14, fontweight='bold')
axes[2].legend(loc='upper right', fontsize=9)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Visualization 4.2: Scenario Comparison

In [ ]:
# Side-by-side comparison of key metrics
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

scenarios = ['No BESS', 'With BESS']
x = np.arange(len(scenarios))
width = 0.35

# Curtailment comparison
curtail_vals = [metrics_no_bess['energy']['total_curtailment_mwh']/1000, 
                metrics_bess['energy']['total_curtailment_mwh']/1000]
bars1 = axes[0, 0].bar(x, curtail_vals, width=0.6, color=['red', 'orange'], alpha=0.7, edgecolor='black')
axes[0, 0].set_ylabel('Energy (GWh)', fontsize=11)
axes[0, 0].set_title('Curtailment', fontsize=12, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(scenarios)
axes[0, 0].grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars1, curtail_vals)):
    axes[0, 0].text(i, val + max(curtail_vals)*0.02, f'{val:,.0f}', 
                     ha='center', fontsize=10, fontweight='bold')

# Grid import comparison
grid_vals = [metrics_no_bess['energy']['total_grid_import_mwh']/1000, 
             metrics_bess['energy']['total_grid_import_mwh']/1000]
bars2 = axes[0, 1].bar(x, grid_vals, width=0.6, color=['gray', 'lightblue'], alpha=0.7, edgecolor='black')
axes[0, 1].set_ylabel('Energy (GWh)', fontsize=11)
axes[0, 1].set_title('Grid Import', fontsize=12, fontweight='bold')
axes[0, 1].set_xticks(x)
axes[0, 1].set_xticklabels(scenarios)
axes[0, 1].grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars2, grid_vals)):
    axes[0, 1].text(i, val + max(grid_vals)*0.02, f'{val:,.0f}', 
                     ha='center', fontsize=10, fontweight='bold')

# Renewable delivery comparison
delivery_vals = [metrics_no_bess['energy']['total_wire_delivery_mwh']/1000, 
                 metrics_bess['energy']['total_wire_delivery_mwh']/1000]
bars3 = axes[1, 0].bar(x, delivery_vals, width=0.6, color=['green', 'darkgreen'], alpha=0.7, edgecolor='black')
axes[1, 0].set_ylabel('Energy (GWh)', fontsize=11)
axes[1, 0].set_title('Renewable Energy Delivered', fontsize=12, fontweight='bold')
axes[1, 0].set_xticks(x)
axes[1, 0].set_xticklabels(scenarios)
axes[1, 0].grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars3, delivery_vals)):
    axes[1, 0].text(i, val + max(delivery_vals)*0.02, f'{val:,.0f}', 
                     ha='center', fontsize=10, fontweight='bold')

# Revenue comparison
revenue_vals = [metrics_no_bess['economics']['renewable_revenue_eur']/1_000_000, 
                metrics_bess['economics']['renewable_revenue_eur']/1_000_000]
bars4 = axes[1, 1].bar(x, revenue_vals, width=0.6, color=['blue', 'darkblue'], alpha=0.7, edgecolor='black')
axes[1, 1].set_ylabel('Revenue (M€)', fontsize=11)
axes[1, 1].set_title('PPA Revenue', fontsize=12, fontweight='bold')
axes[1, 1].set_xticks(x)
axes[1, 1].set_xticklabels(scenarios)
axes[1, 1].grid(True, alpha=0.3, axis='y')
for i, (bar, val) in enumerate(zip(bars4, revenue_vals)):
    axes[1, 1].text(i, val + max(revenue_vals)*0.02, f'€{val:,.1f}M', 
                     ha='center', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

### Visualization 4.3: BESS Performance Analysis

In [ ]:
# Detailed BESS performance visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# SoC distribution
axes[0, 0].hist(ts_bess['bess_soc'] * 100, bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[0, 0].axvline(x=(ts_bess['bess_soc'] * 100).mean(), color='red', linestyle='--', linewidth=2, label='Mean')
axes[0, 0].set_xlabel('State of Charge (%)', fontsize=11)
axes[0, 0].set_ylabel('Frequency', fontsize=11)
axes[0, 0].set_title('BESS SoC Distribution', fontsize=12, fontweight='bold')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Charge/Discharge power distribution
charge_power = ts_bess[ts_bess['bess_charge_mw'] > 0]['bess_charge_mw']
discharge_power = ts_bess[ts_bess['bess_discharge_mw'] > 0]['bess_discharge_mw']
axes[0, 1].hist(charge_power, bins=30, alpha=0.6, color='orange', edgecolor='black', label='Charge')
axes[0, 1].hist(discharge_power, bins=30, alpha=0.6, color='blue', edgecolor='black', label='Discharge')
axes[0, 1].set_xlabel('Power (MW)', fontsize=11)
axes[0, 1].set_ylabel('Frequency', fontsize=11)
axes[0, 1].set_title('BESS Power Distribution', fontsize=12, fontweight='bold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Daily cycling pattern
ts_bess_daily = ts_bess.copy()
ts_bess_daily['date'] = pd.to_datetime(ts_bess_daily['timestamp']).dt.date
daily_charge = ts_bess_daily.groupby('date')['bess_charge_mw'].sum() * timestep_hours
daily_discharge = ts_bess_daily.groupby('date')['bess_discharge_mw'].sum() * timestep_hours
bess_capacity = results_bess['config']['bess_energy_mwh']
daily_cycles = daily_charge / bess_capacity

axes[1, 0].hist(daily_cycles, bins=30, alpha=0.7, color='green', edgecolor='black')
axes[1, 0].axvline(x=daily_cycles.mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {daily_cycles.mean():.2f}')
axes[1, 0].axvline(x=config['bess']['max_daily_cycles'], color='black', linestyle='--', 
                    linewidth=2, label=f'Limit: {config["bess"]["max_daily_cycles"]}')
axes[1, 0].set_xlabel('Daily Cycles', fontsize=11)
axes[1, 0].set_ylabel('Frequency', fontsize=11)
axes[1, 0].set_title('Daily BESS Cycling', fontsize=12, fontweight='bold')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Cumulative energy throughput
cumulative_charge = (ts_bess['bess_charge_mw'] * timestep_hours).cumsum() / 1000  # GWh
cumulative_discharge = (ts_bess['bess_discharge_mw'] * timestep_hours).cumsum() / 1000  # GWh
timesteps_hours = np.arange(len(ts_bess)) * timestep_hours
axes[1, 1].plot(timesteps_hours / 8760, cumulative_charge, label='Cumulative Charge', linewidth=2)
axes[1, 1].plot(timesteps_hours / 8760, cumulative_discharge, label='Cumulative Discharge', linewidth=2)
axes[1, 1].set_xlabel('Time (years)', fontsize=11)
axes[1, 1].set_ylabel('Energy (GWh)', fontsize=11)
axes[1, 1].set_title('Cumulative BESS Throughput', fontsize=12, fontweight='bold')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nBESS Performance Summary:")
print(f"  Total Cycles: {metrics_bess['bess']['total_cycles']:,.1f}")
print(f"  Average Daily Cycles: {daily_cycles.mean():.2f}")
print(f"  Max Daily Cycles: {daily_cycles.max():.2f}")
print(f"  Round-trip Efficiency: {metrics_bess['bess']['roundtrip_efficiency']:.2%}")
print(f"  Capacity Degradation: {metrics_bess['bess']['degradation']:.2%}")
print(f"  Total Energy Charged: {cumulative_charge.iloc[-1]:,.1f} GWh")
print(f"  Total Energy Discharged: {cumulative_discharge.iloc[-1]:,.1f} GWh")

## Conclusion

This notebook has demonstrated the value of battery energy storage for the renewable energy park:

### Key Findings:
1. **Curtailment Reduction**: BESS significantly reduces wasted renewable energy
2. **Grid Import Reduction**: Battery discharge reduces reliance on grid power
3. **Revenue Enhancement**: Increased renewable delivery improves PPA revenue
4. **System Optimization**: Battery operates within constraints while maximizing value

### Export Results:
You can save the simulation results to Google Drive for further analysis:

In [ ]:
# Optional: Export results to CSV
OUTPUT_PATH = f"{GDRIVE_DATA_PATH}/outputs"
import os
os.makedirs(OUTPUT_PATH, exist_ok=True)

# Save No BESS results
ts_no_bess.to_csv(f"{OUTPUT_PATH}/timeseries_no_bess.csv", index=False)
pd.DataFrame([metrics_no_bess]).to_json(f"{OUTPUT_PATH}/metrics_no_bess.json", orient='records', indent=2)

# Save BESS results
ts_bess.to_csv(f"{OUTPUT_PATH}/timeseries_bess.csv", index=False)
pd.DataFrame([metrics_bess]).to_json(f"{OUTPUT_PATH}/metrics_bess.json", orient='records', indent=2)

print(f"Results exported to: {OUTPUT_PATH}")
print("  - timeseries_no_bess.csv")
print("  - metrics_no_bess.json")
print("  - timeseries_bess.csv")
print("  - metrics_bess.json")